In [62]:
import numpy as np 
import pandas as pd 
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
import warnings
import matplotlib.pyplot as plt
import lightgbm as lgb
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [28]:
pheno = pd.read_csv('gpheno_new.csv',index_col=[0])
pheno.head()

,name,progression,training,Patient,Gender,Enrollment,Age.at.diagnosis,Refered,Date.of.bronchoscopy,Age.of.specimen.profiled.months,...,Lab.no.,All.samples..Biopsy.no,Histology,All.samples..Location,Common.samples,Outcone,Cohort,age_group,smoking_group,platform
1,Regressive.1,Regressive,1,105,M,2009.0,67,9-Oct,2010/4/23,7.0,...,UH10-8535,B,MOD,Rul,0.0,Regression,D,60-69,<20,illumina
2,Regressive.2,Regressive,1,104,F,2009.0,71,9-Aug,2009/11/6,3.0,...,UH09-21611,A and B,MOD,LUL stump,0.0,Regression,D,70+,40-69,illumina
3,Regressive.3,Regressive,1,39,M,2005.0,57,5-Oct,2009/4/24,44.0,...,UH09-7377,A,CIS,Trachea Anterior,1.0,Regression,D,60-69,NaN,illumina
4,Regressive.4,Regressive,1,108,M,2010.0,66,10-Oct,2011/4/15,7.0,...,UH11-8074,A,CIS,RLL,1.0,Regression,D,60-69,20-39,illumina
5,Regressive.5,Regressive,1,25,M,2004.0,75,4-Apr,2005/12/16,21.0,...,U05-19490,A,SD,LUL stump,1.0,Regression,D,70+,70+,illumina


In [36]:
data = pd.read_csv('exprSet_DEG100.csv',index_col=[0])
data = data.T
data.head()

,SPNS2,RBAK,ADCYAP1,PAK2,ZNF721,ELK4,AEBP2,IRF7,SEC62,NNT,...,CCT6P1,KLK5,ABCC5,OR14J1,CDC42EP4,SP3,TCEB3CL,XIAP,NME3,COX5B
Regressive.1,7.937416,6.073147,6.216640,7.429954,7.205550,7.307626,5.388207,7.897856,5.884505,6.563337,...,6.030351,6.877620,7.702105,6.032821,9.074438,6.121423,5.066779,5.265389,8.046455,8.196074
Regressive.2,8.150866,5.885627,5.688976,7.993614,7.534645,7.219809,5.363480,7.910522,5.245170,6.263395,...,5.719769,5.855752,8.251707,5.609210,8.576980,6.165575,5.827117,4.966489,7.770363,8.086070
Regressive.3,8.277665,5.916145,6.126428,7.768493,7.315188,7.245931,5.144070,7.894410,5.528479,6.010602,...,5.744348,5.525919,8.257763,5.747189,8.512443,6.488626,5.904026,4.965494,7.885736,8.119360
Regressive.4,9.295856,6.026193,6.014605,7.584193,7.220875,7.009269,5.117852,8.222119,5.248960,6.049178,...,5.907368,7.877984,7.930375,5.509238,9.001206,6.154955,5.993073,4.936723,7.564246,7.793806
Regressive.5,8.866392,5.974143,6.623951,7.672053,7.120102,7.061348,5.043537,8.140913,5.672527,6.333525,...,5.825196,5.726908,8.169803,4.890759,8.902071,6.253743,5.807560,4.978993,7.956618,8.084785


In [37]:
X = data
y = pheno.training

In [59]:
skf = StratifiedKFold(n_splits=10,random_state=5,shuffle=True)
clf = clf = lgb.LGBMClassifier(class_weight={1:19,0:33}, colsample_bytree=0.9,num_leaves=5,reg_alpha=4,reg_lambda=14,
                         min_child_samples=5, importance_type='split')
score1 = []
score2 = []
for train, test in skf.split(X,y):
    X_train,X_test=X.iloc[train],X.iloc[test]
    y_train,y_test=y.iloc[train],y.iloc[test]
    clf.fit(X_train, y_train)
    
    y_pred1 = clf.predict(X_train)
    accuracy1 = np.mean(y_train==y_pred1)
    score1.append(accuracy1)
    
    y_pred2 = clf.predict(X_test)
    accuracy2 = np.mean(y_test==y_pred2)
    score2.append(accuracy2)
score1 = np.array(score1)
score2 = np.array(score2)
print('The accuracy on trainging set is:',np.mean(score1),'The accuracy on test set is:', np.mean(score2))

The accuracy on trainging set is: 1.0 The accuracy on test set is: 0.8
